In [7]:
import Augmentor as aug
import glob
import os
import numpy as np
import cv2 as cv
import PIL
from Augmentor.Operations import Operation
%store -r IMAGE_SIZE


In [8]:
class Lightning(Operation):
    def __init__(self, probability, intensity_low=0.7, intensity_high=1.2):
        Operation.__init__(self, probability)
        self.intensity_low = intensity_low
        self.intensity_high = intensity_high

    def perform_operation(self, images):
        for i, image in enumerate(images):
            image = np.array(image.convert('RGB'))
            row, col, _ = image.shape
            light_intensity = np.random.randint(
                int(self.intensity_low * 100),
                int(self.intensity_high * 100)
            )
            light_intensity /= 100

            gaussian = 100 * np.random.random((row, col, 1))
            gaussian = np.array(gaussian, dtype=np.uint8)
            gaussian = np.concatenate((gaussian, gaussian, gaussian), axis=2)

            image = cv.addWeighted(image, light_intensity, gaussian, 0.25, 0)

            image = PIL.Image.fromarray(image)
            images[i] = image
            
        return images

In [9]:
DIRECTORIES = 'dataset-split/*/*'
folders = []
for f in glob.glob(DIRECTORIES):
    print(f[14:16])
    if os.path.isdir(f):
        if(f[14:16] == "va"):
            folders.append(os.path.abspath(f)+"_va")
        elif(f[14:16] == "tr"):
            folders.append(os.path.abspath(f)+"_tr")
        elif(f[14:16] == "te"):
            folders.append(os.path.abspath(f)+"_te")

print('Classes found {}'.format([os.path.split(x)[1] for x in folders]))


va
va
va
va
va
va
va
va
va
va
va
va
tr
tr
tr
tr
tr
tr
tr
tr
tr
tr
tr
tr
te
te
te
te
te
te
te
te
te
te
te
te
Classes found ['Wart_va', 'Melanocytic naevus_va', 'Lentigo_va', 'Pyogenic Granuloma_va', 'Haemangioma_va', 'Squamous Cell Carcinoma_va', 'Basal Cell Carcinoma_va', 'Dermatofibroma_va', 'seborrhoeic keratosis_va', 'Malignant Melanoma_va', 'Actinic Keratosis_va', 'Intraepithelial Carcinoma_va', 'Wart_tr', 'Melanocytic naevus_tr', 'Lentigo_tr', 'Pyogenic Granuloma_tr', 'Haemangioma_tr', 'Squamous Cell Carcinoma_tr', 'Basal Cell Carcinoma_tr', 'Dermatofibroma_tr', 'seborrhoeic keratosis_tr', 'Malignant Melanoma_tr', 'Actinic Keratosis_tr', 'Intraepithelial Carcinoma_tr', 'Wart_te', 'Melanocytic naevus_te', 'Lentigo_te', 'Pyogenic Granuloma_te', 'Haemangioma_te', 'Squamous Cell Carcinoma_te', 'Basal Cell Carcinoma_te', 'Dermatofibroma_te', 'seborrhoeic keratosis_te', 'Malignant Melanoma_te', 'Actinic Keratosis_te', 'Intraepithelial Carcinoma_te']


In [10]:
pipelines = {}
for folder in folders:
    print(folder[:-2])
    pipelines[os.path.split(folder)[1]] = \
        (aug.Pipeline(source_directory=folder[:-3],
            output_directory='.',
            save_format='jpg'
            )
        )

classes_count = []
for p in pipelines.values():
    print("Class '{}' has {} samples".format(
        p.augmentor_images[0].class_label,
        len(p.augmentor_images)
    ))
    classes_count.append(len(p.augmentor_images))

print(classes_count)

/home/matheuscosta/Workbench/XAI/dataset-split/val/Wart_
Initialised with 10 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset-split/val/Wart/../home/matheuscosta/Workbench/XAI/dataset-split/val/Melanocytic naevus_
Initialised with 33 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset-split/val/Melanocytic naevus/../home/matheuscosta/Workbench/XAI/dataset-split/val/Lentigo_
Initialised with 8 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset-split/val/Lentigo/../home/matheuscosta/Workbench/XAI/dataset-split/val/Pyogenic Granuloma_
Initialised with 2 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset-split/val/Pyogenic Granuloma/../home/matheuscosta/Workbench/XAI/dataset-split/val/Haemangioma_
Initialised with 11 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset-split/val/Haemangioma/../home/matheuscosta/Workbench/XAI/dataset-split/val/

In [11]:
lightning = Lightning(probability=0.3)

for p in pipelines.values():
    p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
    # p.zoom_random(probability=0.4, percentage_area=0.9)
    p.flip_random(probability=0.5)
    p.flip_left_right(probability=0.7)
    p.flip_top_bottom(probability=0.5)
    p.shear(probability=0.5, max_shear_left=10, max_shear_right=10)
    p.random_distortion(probability=0.8, grid_width=5, grid_height=5, magnitude=15)
    p.add_operation(lightning)
    p.resize(probability=1.0, width=IMAGE_SIZE, height=IMAGE_SIZE)

In [12]:
SAME_SAMPLING = False
MULTIPLIER = 29

for p in pipelines.values():
    if SAME_SAMPLING:
        diff = max(classes_count) - len(p.augmentor_images)
        p.sample((len(p.augmentor_images) + diff)*MULTIPLIER + diff)
    else:
        p.sample(len(p.augmentor_images)*MULTIPLIER)

Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03BC5AA580>: 100%|██████████| 290/290 [00:00<00:00, 570.93 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03AC118160>: 100%|██████████| 957/957 [00:01<00:00, 544.46 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03BDE5F790>: 100%|██████████| 232/232 [00:00<00:00, 512.17 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03BC5947F0>: 100%|██████████| 58/58 [00:00<00:00, 589.27 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03BC5A8880>: 100%|██████████| 319/319 [00:00<00:00, 506.77 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F03BC4DE700>: 100%|██████████| 928/928 [00:01<00:00, 522.14 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F038C683910>: 100%|██████████| 2001/2001 [00:03<00:00, 523.16 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F036D